한 환자의 검진 결과를 test_time과 inter로 구분해서 나눠주는 함수
* test_time : n번의 결과만 가지게 나눠줌. ex) 총 7번 검사 중에 test_time=4라면 1234, 2345, 3456, ...
* inter : 검진 날짜 사이의 간격을 정해줌. 
    * test_time=3, inter=360이라면 검사를 360일 안에 한 데이터가 연속적으로 3개인 것들만 남을 것

In [18]:
import pandas as pd
import os
import shutil

# DataName = input("data = (ex. combine, pusan) ")
# test_time = input('test_time = ')
# eye = input("eye = (ex.OS, OD, None) ")
# # eye = 'None'
# inter = input("inter = (ex. interval 6 month = 180) ")

test_time = 4
# eye = None
# eye = 'None'
inter = 360


csv = pd.read_csv('./data/want_data.csv')
csv['Exam Date'] = pd.to_datetime(csv['Exam Date'].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:]))

# pusan = pd.read_csv('./data/pusan.csv',index_col=0)


# FileName = str(DataName) + str(test_time) + str(eye) + str(inter)  

# if DataName == 'combine':
#     csv = combine
# else:
#     csv = pusan    

# if eye == 'OS':
#     csv = csv[csv.Eye == eye]
# elif eye == 'OD':
#     csv = csv[csv.Eye == eye]
# else:
#     pass
    
csv = csv.reset_index(drop=True)    

def inter_idx(csvs,inters):
    data = csvs.copy()
    
    data = data.sort_values(by=['PID', 'Exam Date'], axis=0)
    data['Exam Date'] = pd.to_datetime(data['Exam Date'])
    
    idx = []
    for x in data['PID'].unique():
        temp = data.loc[data['PID']==x].copy()
        temp['Exam Date'] = (temp['Exam Date'] - temp['Exam Date'].iloc[0]).dt.days

        ind_list = []
        all_set = []
        real_ind = []
        for x in temp['Exam Date']:
            inters = [x]
            for y in temp['Exam Date']:
                if int(inter) - 30 <= y-x <= int(inter) + 30 :
                    inters.append(y)
                    x = y 
            all_set.append(inters)

        for a in all_set:
            if len(a)>=int(test_time):
                ind_list.append(a)
                for k in ind_list:
                    if len(k)>int(test_time):
                        ind_list.remove(k)
                        for q in range(len(k)-test_time):
                            ind_list.append(k[0+q:test_time+q])

        for ind in ind_list : 
            if ind not in real_ind :
                real_ind.append(ind)

        for real in real_ind :
        #         line = a
            idx += temp[temp['Exam Date'].isin(real) == True].index.tolist()
    
    return idx

csv_idx = inter_idx(csv,inter)
csv = csv.iloc[csv_idx]

print(csv.shape)
csv['new_pid'] = ''
for x in csv['PID'].unique() : 
    new_csv = csv.reset_index().reset_index()
    repe_list = new_csv.loc[new_csv['PID']==x]['level_0'].tolist()

    for i in range(int(len(repe_list)/test_time)):
        csv['new_pid'][repe_list[0]+test_time*i:repe_list[0]+test_time+test_time*i] = str(x)+'_'+str(i)

def date2days(csvs):
    data = csvs.copy()
    
#     data = data.sort_values(by=['new_pid', 'Exam Date'], axis=0, reverse=True)
#     data['Exam Date'] = pd.to_datetime(data['Exam Date'])
    
    df_list = []
    for x in data['new_pid'].unique():
        temp = data[data['new_pid']==x].copy()
        temp['Exam Date'] = (temp['Exam Date'] - temp['Exam Date'].iloc[-2]).dt.days
        df_list.append(temp)
    
    return pd.concat(df_list)



new_file = date2days(csv)

print(len(csv['new_pid'].unique()))

FileName = 'Data_per_time'
new_file.to_csv(FileName + ".csv", index=False)

(128, 394)


<ipython-input-18-6a3fec55f447>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv['new_pid'][repe_list[0]+test_time*i:repe_list[0]+test_time+test_time*i] = str(x)+'_'+str(i)
C:\Users\82103\anaconda3\envs\glaucoma\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-18-6a3fec55f447>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv[

<ipython-input-18-6a3fec55f447>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv['new_pid'][repe_list[0]+test_time*i:repe_list[0]+test_time+test_time*i] = str(x)+'_'+str(i)
C:\Users\82103\anaconda3\envs\glaucoma\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


32


In [28]:
pd.read_csv('./data/want_data.csv')[pd.read_csv('./data/want_data.csv')['PID']==1]

PID  AGE Eye  Exam Date  Pupil diameter  False positive percent  \
0    1   51  OS   20101119             6.4                       1   
1    1   52  OS   20111111             0.0                       1   
2    1   53  OS   20121031             3.9                       0   
3    1   54  OS   20131025             3.7                      15   
4    1   55  OS   20140926             0.0                       5   
5    1   56  OS   20151006             0.0                      12   

   False negative percent  Fixation loss  Fixation trial  Exam duration(min)  \
0                       0              0              13                   4   
1                       0              1              13                   4   
2                       0              2              14                   4   
3                       7              1              15                   5   
4                       4              2              14                   4   
5                       0              0              13                   4   

   ...  mythv_45  mythv_46  mythv_47  mythv_48  mythv_49  mythv_50  mythv_51  \
0  ...        29        31        31        31        30        30        27   
1  ...        32        32        27        29        30        27        29   
2  ...        29        31        30        28        30        28        31   
3  ...        28        29        30        31        33        31        29   
4  ...        33        32        30        30        32        31        31   
5  ...        31        31        28        31        32        30        29   

   mythv_52  mythv_53  mythv_54  
0        30        30        30  
1        30        31        29  
2        30        29        29  
3        30        29        29  
4        32        30        30  
5        31        24        27  

[6 rows x 394 columns]

In [29]:
temp[temp['PID']==1]

PID  AGE Eye  Exam Date  Pupil diameter  False positive percent  \
0     1   51  OS       -712             6.4                       1   
1     1   52  OS       -355             0.0                       1   
2     1   53  OS          0             3.9                       0   
3     1   54  OS        359             3.7                      15   
4     1   52  OS       -714             0.0                       1   
5     1   53  OS       -359             3.9                       0   
6     1   54  OS          0             3.7                      15   
7     1   55  OS        336             0.0                       5   
8     1   53  OS       -695             3.9                       0   
9     1   54  OS       -336             3.7                      15   
10    1   55  OS          0             0.0                       5   
11    1   56  OS        375             0.0                      12   

    False negative percent  Fixation loss  Fixation trial  Exam duration(min)  \
0                        0              0              13                   4   
1                        0              1              13                   4   
2                        0              2              14                   4   
3                        7              1              15                   5   
4                        0              1              13                   4   
5                        0              2              14                   4   
6                        7              1              15                   5   
7                        4              2              14                   4   
8                        0              2              14                   4   
9                        7              1              15                   5   
10                       4              2              14                   4   
11                       0              0              13                   4   

    ...  mythv_46  mythv_47  mythv_48  mythv_49  mythv_50  mythv_51  mythv_52  \
0   ...        31        31        31        30        30        27        30   
1   ...        32        27        29        30        27        29        30   
2   ...        31        30        28        30        28        31        30   
3   ...        29        30        31        33        31        29        30   
4   ...        32        27        29        30        27        29        30   
5   ...        31        30        28        30        28        31        30   
6   ...        29        30        31        33        31        29        30   
7   ...        32        30        30        32        31        31        32   
8   ...        31        30        28        30        28        31        30   
9   ...        29        30        31        33        31        29        30   
10  ...        32        30        30        32        31        31        32   
11  ...        31        28        31        32        30        29        31   

    mythv_53  mythv_54  new_pid  
0         30        30      1_0  
1         31        29      1_0  
2         29        29      1_0  
3         29        29      1_0  
4         31        29      1_1  
5         29        29      1_1  
6         29        29      1_1  
7         30        30      1_1  
8         29        29      1_2  
9         29        29      1_2  
10        30        30      1_2  
11        24        27      1_2  

[12 rows x 395 columns]

In [20]:
import pandas as pd
pd.read_csv('./Data_per_time.csv')

PID  AGE Eye  Exam Date  Pupil diameter  False positive percent  \
0      1   51  OS       -712             6.4                       1   
1      1   52  OS       -355             0.0                       1   
2      1   53  OS          0             3.9                       0   
3      1   54  OS        359             3.7                      15   
4      1   52  OS       -714             0.0                       1   
..   ...  ...  ..        ...             ...                     ...   
123  102   55  OS        354             6.0                       4   
124  103   52  OD       -685             5.7                       6   
125  103   53  OD       -343             5.6                       7   
126  103   54  OD          0             5.0                      10   
127  103   55  OD        354             5.8                       4   

     False negative percent  Fixation loss  Fixation trial  \
0                         0              0              13   
1                         0              1              13   
2                         0              2              14   
3                         7              1              15   
4                         0              1              13   
..                      ...            ...             ...   
123                       1              0              17   
124                       7              2              14   
125                       0              2              15   
126                       0              3              14   
127                       3              2              15   

     Exam duration(min)  ...  mythv_46  mythv_47  mythv_48  mythv_49  \
0                     4  ...        31        31        31        30   
1                     4  ...        32        27        29        30   
2                     4  ...        31        30        28        30   
3                     5  ...        29        30        31        33   
4                     4  ...        32        27        29        30   
..                  ...  ...       ...       ...       ...       ...   
123                   6  ...        25        26        27        28   
124                   4  ...        31        29        31        29   
125                   5  ...        29        28        29        27   
126                   4  ...        29        29        30        30   
127                   5  ...        28        28        31        27   

     mythv_50  mythv_51  mythv_52  mythv_53  mythv_54  new_pid  
0          30        27        30        30        30      1_0  
1          27        29        30        31        29      1_0  
2          28        31        30        29        29      1_0  
3          31        29        30        29        29      1_0  
4          27        29        30        31        29      1_1  
..        ...       ...       ...       ...       ...      ...  
123        26        26        28        26        28    102_0  
124        27        30        27        33        35    103_0  
125        27        26        29        28        28    103_0  
126        29        30        29        30        31    103_0  
127        27        28        32        33        25    103_0  

[128 rows x 395 columns]

In [21]:
temp = pd.read_csv('./Data_per_time.csv')

In [22]:
len(temp[temp['PID']==i])

0

In [27]:
temp[temp['PID']==1]

PID  AGE Eye  Exam Date  Pupil diameter  False positive percent  \
0     1   51  OS       -712             6.4                       1   
1     1   52  OS       -355             0.0                       1   
2     1   53  OS          0             3.9                       0   
3     1   54  OS        359             3.7                      15   
4     1   52  OS       -714             0.0                       1   
5     1   53  OS       -359             3.9                       0   
6     1   54  OS          0             3.7                      15   
7     1   55  OS        336             0.0                       5   
8     1   53  OS       -695             3.9                       0   
9     1   54  OS       -336             3.7                      15   
10    1   55  OS          0             0.0                       5   
11    1   56  OS        375             0.0                      12   

    False negative percent  Fixation loss  Fixation trial  Exam duration(min)  \
0                        0              0              13                   4   
1                        0              1              13                   4   
2                        0              2              14                   4   
3                        7              1              15                   5   
4                        0              1              13                   4   
5                        0              2              14                   4   
6                        7              1              15                   5   
7                        4              2              14                   4   
8                        0              2              14                   4   
9                        7              1              15                   5   
10                       4              2              14                   4   
11                       0              0              13                   4   

    ...  mythv_46  mythv_47  mythv_48  mythv_49  mythv_50  mythv_51  mythv_52  \
0   ...        31        31        31        30        30        27        30   
1   ...        32        27        29        30        27        29        30   
2   ...        31        30        28        30        28        31        30   
3   ...        29        30        31        33        31        29        30   
4   ...        32        27        29        30        27        29        30   
5   ...        31        30        28        30        28        31        30   
6   ...        29        30        31        33        31        29        30   
7   ...        32        30        30        32        31        31        32   
8   ...        31        30        28        30        28        31        30   
9   ...        29        30        31        33        31        29        30   
10  ...        32        30        30        32        31        31        32   
11  ...        31        28        31        32        30        29        31   

    mythv_53  mythv_54  new_pid  
0         30        30      1_0  
1         31        29      1_0  
2         29        29      1_0  
3         29        29      1_0  
4         31        29      1_1  
5         29        29      1_1  
6         29        29      1_1  
7         30        30      1_1  
8         29        29      1_2  
9         29        29      1_2  
10        30        30      1_2  
11        24        27      1_2  

[12 rows x 395 columns]

In [25]:
for i in temp['PID'].unique():
    if len(temp[temp['PID']==i])>4 :
        print(i)

1
2
3
4
23
29
30


In [26]:
for i in range(temp.shape[0]):
    temp[temp['PID'] == temp['PID'].unique()[10]].shape[0] != 3